In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from datetime import date



symbols  = [ 'SPY', "QQQ", "VGK", "SCZ", "EWJ", "EEM", #eq
           'REM', "VNQ", "RWX", # re
           'TIP', "IEF", "TLT", "BWX", #fi
           'DBC', "GLD", #como
           'twtr', 'intc', 'tsm',"VWUSX", "Amc"]

all_stocks = pd.DataFrame()
 
for symbol in symbols:
    tmp_close = yf.download(symbol, 
                      start='2022-01-01', 
                      end= date.today(),
                      interval = "1d",
                      threads = True,
                      progress=False)['Adj Close']
    all_stocks = pd.concat([all_stocks, tmp_close], axis=1)
 
all_stocks.columns=symbols
all_stocks

,SPY,QQQ,VGK,SCZ,EWJ,EEM,REM,VNQ,RWX,TIP,IEF,TLT,BWX,DBC,GLD,twtr,intc,tsm,VWUSX,Amc
2021-12-31 00:00:00,474.959991,397.850006,68.239998,73.089996,66.959999,48.849998,34.509998,116.010002,35.599998,128.751770,114.889671,147.983932,28.247410,20.780001,170.960007,43.220001,51.110657,120.309998,65.029999,27.200001
2022-01-03 00:00:00,477.709991,401.679993,68.889999,73.430000,67.029999,49.200001,35.189999,115.099998,35.639999,128.014343,113.820694,144.099350,27.907681,20.860001,168.330002,42.660000,52.807728,128.800003,65.510002,26.520000
2022-01-04 00:00:00,477.549988,396.470001,68.980003,73.650002,67.910004,49.029999,35.310001,114.959999,35.570000,127.526039,113.760757,143.500168,27.907681,21.110001,169.570007,40.849998,52.738258,133.399994,64.040001,25.490000
2022-01-05 00:00:00,468.380005,384.290009,68.360001,72.919998,67.599998,48.230000,34.680000,111.669998,34.980000,126.489647,113.281212,142.721252,27.787775,21.020000,169.059998,39.500000,53.462738,127.059998,61.439999,22.750000
2022-01-06 00:00:00,467.940002,384.019989,68.110001,72.360001,67.099998,48.450001,34.790001,111.769997,34.860001,125.941551,113.001480,143.090744,27.737816,21.200001,166.990005,39.590000,53.601677,128.470001,61.209999,22.459999
2022-01-07 00:00:00,466.089996,379.859985,68.449997,72.290001,66.669998,48.889999,34.970001,111.029999,34.790001,125.792076,112.631836,142.062164,27.807760,21.240000,167.750000,39.669998,53.035988,123.500000,60.480000,22.990000
2022-01-10 00:00:00,465.510010,380.109985,67.519997,71.349998,66.510002,48.889999,34.930000,110.389999,34.610001,125.951515,112.601868,142.411697,27.837738,21.150000,168.259995,39.970001,54.792606,125.010002,60.509998,22.780001
2022-01-11 00:00:00,469.750000,385.820007,68.360001,72.260002,67.029999,50.020000,35.290001,110.559998,34.689999,126.828461,112.871613,143.360367,27.887697,21.559999,170.289993,40.660000,55.487316,129.169998,61.590000,22.790001
2022-01-12 00:00:00,471.019989,387.350006,68.989998,73.099998,67.989998,50.849998,35.310001,110.690002,34.849998,126.011307,112.891586,142.811127,28.117512,21.730000,170.740005,40.250000,55.318604,132.229996,61.610001,22.719999
2022-01-13 00:00:00,464.529999,377.660004,68.480003,72.540001,67.309998,50.130001,35.139999,110.470001,34.689999,126.041206,113.251244,144.079361,28.197449,21.500000,170.160004,38.700001,54.524647,139.190002,59.349998,20.660000


In [5]:
returns = np.log(all_stocks/all_stocks.shift(1))
returns.fillna(0)
returns = returns.iloc[1: , :]
returns

,SPY,QQQ,VGK,SCZ,EWJ,EEM,REM,VNQ,RWX,TIP,IEF,TLT,BWX,DBC,GLD,twtr,intc,tsm,VWUSX,Amc
2022-01-03 00:00:00,0.005773,0.009581,0.009480,0.004641,0.001045,0.007139,0.019513,-0.007875,0.001123,-0.005744,-0.009348,-0.026601,-0.012100,0.003842,-0.015503,-0.013042,0.032665,0.068189,0.007354,-0.025318
2022-01-04 00:00:00,-0.000335,-0.013055,0.001306,0.002992,0.013043,-0.003461,0.003404,-0.001217,-0.001966,-0.003822,-0.000527,-0.004167,0.000000,0.011913,0.007340,-0.043355,-0.001316,0.035091,-0.022695,-0.039613
2022-01-05 00:00:00,-0.019389,-0.031203,-0.009029,-0.009961,-0.004575,-0.016451,-0.018003,-0.029036,-0.016726,-0.008160,-0.004224,-0.005443,-0.004306,-0.004273,-0.003012,-0.033606,0.013644,-0.048693,-0.041447,-0.113721
2022-01-06 00:00:00,-0.000940,-0.000703,-0.003664,-0.007709,-0.007424,0.004551,0.003167,0.000895,-0.003436,-0.004343,-0.002472,0.002586,-0.001800,0.008527,-0.012320,0.002276,0.002595,0.011036,-0.003751,-0.012829
2022-01-07 00:00:00,-0.003961,-0.010892,0.004979,-0.000968,-0.006429,0.009041,0.005161,-0.006643,-0.002010,-0.001188,-0.003277,-0.007214,0.002518,0.001885,0.004541,0.002019,-0.010610,-0.039454,-0.011998,0.023323
2022-01-10 00:00:00,-0.001245,0.000658,-0.013680,-0.013088,-0.002403,0.000000,-0.001145,-0.005781,-0.005187,0.001267,-0.000266,0.002457,0.001077,-0.004246,0.003036,0.007534,0.032585,0.012153,0.000496,-0.009176
2022-01-11 00:00:00,0.009067,0.014910,0.012364,0.012673,0.007788,0.022850,0.010254,0.001539,0.002309,0.006938,0.002393,0.006639,0.001793,0.019200,0.011992,0.017116,0.012599,0.032736,0.017691,0.000439
2022-01-12 00:00:00,0.002700,0.003958,0.009174,0.011558,0.014220,0.016457,0.000567,0.001175,0.004602,-0.006464,0.000177,-0.003839,0.008207,0.007854,0.002639,-0.010135,-0.003045,0.023413,0.000325,-0.003076
2022-01-13 00:00:00,-0.013874,-0.025334,-0.007420,-0.007690,-0.010052,-0.014260,-0.004826,-0.001990,-0.004602,0.000237,0.003181,0.008841,0.002839,-0.010641,-0.003403,-0.039270,-0.014456,0.051297,-0.037372,-0.095046
2022-01-14 00:00:00,0.000409,0.006203,-0.002778,-0.004698,-0.001189,-0.000399,-0.003421,-0.009093,0.000576,-0.004517,-0.006905,-0.015225,-0.004973,0.012939,-0.002884,-0.006741,0.013739,0.010506,0.001515,-0.004366


In [10]:
all_stocks[-1::].values[0]

array([429.57000732, 338.07998657,  63.70000076,  66.91000366,
        63.11999893,  48.02999878,  31.55999947, 101.63999939,
        33.49000168, 124.51999664, 111.20999908, 138.58999634,
        27.32999992,  23.51000023, 177.49000549,  32.93000031,
        44.68999863, 116.91000366,  51.93000031,  16.46999931])

In [55]:
from scipy.optimize import minimize, Bounds

lamb = 0.01 #Faire attention car sinon il n'y a pas de mouvements de position
alpha = 200
Money = 0
Pos = []

# p0 = nombre de chaque stocks qui correspond a un investissement equitable par stocks
p0 = [Money/(returns.shape[1]*all_stocks[-1::].values[0][i]) for i in range(returns.shape[1])]

Pos.append(np.array(p0))

def objective_corr(pos):
    pos = np.array(pos)
    s = pos@np.corrcoef(returns.iloc[1: , :].fillna(0).T)@pos - alpha*returns.mean()@pos + \
    TC(Pos, pos)
    
    print('\033[92m'+"Cout rajoutés de transaction: "+str(TC(Pos,pos))+'\033[0m')
    print('\033[93m'+"Cout de mean: " + str(alpha*returns.mean()@pos) + '\033[0m')
    print('\033[94m'+"Somme des positions: "+str(all_stocks[-1::].values[0]@np.array(pos).T)+'\033[0m')
    print(f"Les valeurs  du vecteur poids sont:{pos.round(3)}")
    print(f"Les valeurs de la fonction objective:{s.round(3)}")
    return s

def TC(Pos,pos):
    return lamb*(all_stocks[-1::].values[0]@np.abs(np.array(pos)-np.array(Pos[-1]))) + \
    all_stocks[-1::].values[0]@np.array(Pos[-1]-pos).T

cons = ({"type":"ineq", "fun": lambda x: Money - all_stocks[-1::].values[0]@np.array(x).T }, 
        # The positions must invest no more than the initial money.
        # This constraints says that the inequalities (ineq) must be non-negative.
        #{"type": "ineq", "fun": lambda x: -TC(Pos,x)},
        # The expected daily return of our portfolio and we want to be at greater than 0.003
        #{"type": "ineq", "fun": lambda x: -np.sum(returns.mean()*x)+1})
       )
 
 
#Every stock can get any position from 0 to Money
bounds = Bounds(0, Money)
 
p = minimize(objective_corr, p0, 
             method = "SLSQP", 
             # bounds= bounds,
             constraints= cons, options={'maxiter': 300, 'ftol': 1e-09})


Pos.append(p.x)
p

Cout rajoutés de transaction: 0.0
Cout de mean: 0.0
Somme des positions: 0.0
Les valeurs  du vecteur poids sont:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Les valeurs de la fonction objective:0.0
Cout rajoutés de transaction: -6.337081003948697e-06
Cout de mean: -8.552912862595033e-09
Somme des positions: 6.401091923180502e-06
Les valeurs  du vecteur poids sont:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Les valeurs de la fonction objective:-0.0
Cout rajoutés de transaction: -4.987406532563909e-06
Cout de mean: -1.3861703107584396e-08
Somme des positions: 5.037784376327181e-06
Les valeurs  du vecteur poids sont:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Les valeurs de la fonction objective:-0.0
Cout rajoutés de transaction: -9.397119396226117e-07
Cout de mean: -5.862227280103129e-09
Somme des positions: 9.492039794167795e-07
Les valeurs  du vecteur poids sont:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Les valeur

Cout de mean: 1.065362823008428
Somme des positions: 1.984391394316276e-06
Les valeurs  du vecteur poids sont:[ 0.034 -0.009 -0.018 -0.026 -0.014  0.001 -0.033 -0.039 -0.021  0.008
  0.006 -0.003 -0.009  0.058  0.049 -0.113 -0.05   0.009 -0.089 -0.216]
Les valeurs de la fonction objective:-0.273
Cout rajoutés de transaction: 0.5520006447263801
Cout de mean: 1.065362815689812
Somme des positions: 4.278872273388856e-06
Les valeurs  du vecteur poids sont:[ 0.034 -0.009 -0.018 -0.026 -0.014  0.001 -0.033 -0.039 -0.021  0.008
  0.006 -0.003 -0.009  0.058  0.049 -0.113 -0.05   0.009 -0.089 -0.216]
Les valeurs de la fonction objective:-0.273
Cout rajoutés de transaction: 0.5520027674832954
Cout de mean: 1.0653627893442248
Somme des positions: 2.1247603338991894e-06
Les valeurs  du vecteur poids sont:[ 0.034 -0.009 -0.018 -0.026 -0.014  0.001 -0.033 -0.039 -0.021  0.008
  0.006 -0.003 -0.009  0.058  0.049 -0.113 -0.05   0.009 -0.089 -0.216]
Les valeurs de la fonction objective:-0.273
Cout rajo

Somme des positions: 1.6444162334039447e-06
Les valeurs  du vecteur poids sont:[-0.028  0.063  0.026  0.007  0.03   0.031 -0.036 -0.021 -0.003  0.024
  0.012 -0.007 -0.     0.141  0.061 -0.252 -0.086  0.01  -0.172 -0.546]
Les valeurs de la fonction objective:-0.607
Cout rajoutés de transaction: 0.9284939880894751
Cout de mean: 2.316315126592625
Somme des positions: 1.5879407619934227e-06
Les valeurs  du vecteur poids sont:[-0.028  0.063  0.026  0.007  0.03   0.031 -0.036 -0.021 -0.003  0.024
  0.012 -0.007 -0.     0.141  0.061 -0.252 -0.086  0.01  -0.172 -0.546]
Les valeurs de la fonction objective:-0.607
Cout rajoutés de transaction: 0.9284942106994146
Cout de mean: 2.3163151301799045
Somme des positions: 1.3630822373045248e-06
Les valeurs  du vecteur poids sont:[-0.028  0.063  0.026  0.007  0.03   0.031 -0.036 -0.021 -0.003  0.024
  0.012 -0.007 -0.     0.141  0.061 -0.252 -0.086  0.01  -0.172 -0.546]
Les valeurs de la fonction objective:-0.607
Cout rajoutés de transaction: 0.9284944

Cout rajoutés de transaction: 3.06332406653937
Cout de mean: 5.524949313723458
Somme des positions: 6.69881559822727e-06
Les valeurs  du vecteur poids sont:[-0.05  -0.155  0.26   0.183  0.199  0.341  0.163  0.168  0.245 -0.072
 -0.174  0.205  0.042  0.341  0.041 -0.498  0.031  0.161 -0.132 -1.688]
Les valeurs de la fonction objective:0.118
Cout rajoutés de transaction: 1.4965483601482665
Cout de mean: 4.348887165192247
Somme des positions: 3.7647438633214847e-06
Les valeurs  du vecteur poids sont:[-0.002 -0.031  0.175  0.125  0.151  0.186  0.07   0.122  0.139 -0.027
 -0.081  0.044  0.04   0.276 -0.023 -0.438 -0.05   0.033 -0.195 -1.237]
Les valeurs de la fonction objective:-1.168
Cout rajoutés de transaction: 1.2988331252924723
Cout de mean: 4.13192483484505
Somme des positions: 3.223460363521724e-06
Les valeurs  du vecteur poids sont:[ 0.007 -0.008  0.16   0.115  0.142  0.157  0.053  0.113  0.12  -0.018
 -0.064  0.015  0.04   0.264 -0.035 -0.427 -0.065  0.009 -0.207 -1.154]
Les valeur

Cout rajoutés de transaction: 1.5673789571125827
Cout de mean: 4.713781966413901
Somme des positions: 2.1812322064107548e-06
Les valeurs  du vecteur poids sont:[-0.008 -0.019  0.177  0.107  0.119  0.292  0.127  0.091  0.192 -0.045
 -0.139 -0.034  0.014  0.276  0.05  -0.413  0.023  0.017 -0.11  -1.42 ]
Les valeurs de la fonction objective:-1.466
Cout rajoutés de transaction: 1.5673766855765123
Cout de mean: 4.713781959095284
Somme des positions: 4.475713085483335e-06
Les valeurs  du vecteur poids sont:[-0.008 -0.019  0.177  0.107  0.119  0.292  0.127  0.091  0.192 -0.045
 -0.139 -0.034  0.014  0.276  0.05  -0.413  0.023  0.017 -0.11  -1.42 ]
Les valeurs de la fonction objective:-1.466
Cout rajoutés de transaction: 1.5673788083334275
Cout de mean: 4.713781932749697
Somme des positions: 2.321601145993668e-06
Les valeurs  du vecteur poids sont:[-0.008 -0.019  0.177  0.107  0.119  0.292  0.127  0.091  0.192 -0.045
 -0.139 -0.034  0.014  0.276  0.05  -0.413  0.023  0.017 -0.11  -1.42 ]
Les v

...

Cout rajoutés de transaction: 1.327673042788496
Cout de mean: 6.7918424474095085
Somme des positions: -2.3585524889513376e-06
Les valeurs  du vecteur poids sont:[-0.     0.    -0.     0.    -0.     1.379 -0.     0.     0.     0.
  0.    -0.149 -0.     0.006  0.    -0.262  0.    -0.     0.    -2.253]
Les valeurs de la fonction objective:-2.732
Cout rajoutés de transaction: 1.3276707712524256
Cout de mean: 6.791842440090893
Somme des positions: -6.407160987875749e-08
Les valeurs  du vecteur poids sont:[-0.     0.    -0.     0.    -0.     1.379 -0.     0.     0.     0.
  0.    -0.149 -0.     0.006  0.    -0.262  0.    -0.     0.    -2.253]
Les valeurs de la fonction objective:-2.732
Cout rajoutés de transaction: 1.327672894009341
Cout de mean: 6.791842413745306
Somme des positions: -2.2181835493684243e-06
Les valeurs  du vecteur poids sont:[-0.     0.    -0.     0.    -0.     1.379 -0.     0.     0.     0.
  0.    -0.149 -0.     0.006  0.    -0.262  0.    -0.     0.    -2.253]
Les valeurs

Somme des positions: -1.7713709731260725e-06
Les valeurs  du vecteur poids sont:[-0.     0.    -0.     0.    -0.     1.379 -0.     0.     0.     0.
  0.    -0.149 -0.     0.006  0.    -0.262  0.    -0.     0.    -2.253]
Les valeurs de la fonction objective:-2.732
Cout rajoutés de transaction: 1.3276728069866315
Cout de mean: 6.791842418053565
Somme des positions: -1.9962294978149704e-06
Les valeurs  du vecteur poids sont:[-0.     0.    -0.     0.    -0.     1.379 -0.     0.     0.     0.
  0.    -0.149 -0.     0.006  0.    -0.262  0.    -0.     0.    -2.253]
Les valeurs de la fonction objective:-2.732
Cout rajoutés de transaction: 1.327673040548912
Cout de mean: 6.791842411886183
Somme des positions: -2.241651612445826e-06
Les valeurs  du vecteur poids sont:[-0.     0.    -0.     0.    -0.     1.379 -0.     0.     0.     0.
  0.    -0.149 -0.     0.006  0.    -0.262  0.    -0.     0.    -2.253]
Les valeurs de la fonction objective:-2.732
Cout rajoutés de transaction: 1.3276720161238833

Cout rajoutés de transaction: 1.3295337927300261
Cout de mean: 6.791832811868628
Somme des positions: 2.6256518725481515e-06
Les valeurs  du vecteur poids sont:[ 0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00  0.000e+00  1.381e+00
  0.000e+00 -0.000e+00 -2.000e-03 -0.000e+00 -0.000e+00 -1.490e-01
  0.000e+00  7.000e-03 -0.000e+00 -2.620e-01 -0.000e+00  0.000e+00
 -0.000e+00 -2.252e+00]
Les valeurs de la fonction objective:-2.73
Cout rajoutés de transaction: 1.3278533958236325
Cout de mean: 6.791841464265955
Somme des positions: -2.1779628526369388e-06
Les valeurs  du vecteur poids sont:[-0.     0.    -0.     0.    -0.     1.379  0.    -0.    -0.     0.
  0.    -0.149 -0.     0.006  0.    -0.262  0.    -0.     0.    -2.253]
Les valeurs de la fonction objective:-2.731
Cout rajoutés de transaction: 1.3276910696821416
Cout de mean: 6.791842329505688
Somme des positions: -2.6583243388379058e-06
Les valeurs  du vecteur poids sont:[-0.     0.    -0.     0.    -0.     1.379 -0.     0.    -0.     0

Les valeurs  du vecteur poids sont:[-0.     0.    -0.     0.    -0.     1.379 -0.     0.     0.     0.
  0.    -0.149 -0.     0.006  0.    -0.262  0.    -0.     0.    -2.253]
Les valeurs de la fonction objective:-2.732
Cout rajoutés de transaction: 1.327671909760401
Cout de mean: 6.7918424057030755
Somme des positions: -1.0540839524878724e-06
Les valeurs  du vecteur poids sont:[-0.     0.    -0.     0.    -0.     1.379 -0.     0.     0.     0.
  0.    -0.149 -0.     0.006  0.    -0.262  0.    -0.     0.    -2.253]
Les valeurs de la fonction objective:-2.732
Cout rajoutés de transaction: 1.3276714645435486
Cout de mean: 6.791842402890434
Somme des positions: -6.460901999275851e-07
Les valeurs  du vecteur poids sont:[-0.     0.    -0.     0.    -0.     1.379 -0.     0.     0.     0.
  0.    -0.149 -0.     0.006  0.    -0.262  0.    -0.     0.    -2.253]
Les valeurs de la fonction objective:-2.732
Cout rajoutés de transaction: 1.3276731390257208
Cout de mean: 6.7918424056635
Somme des pos

     fun: -2.7316554839324936
     jac: array([-434.23986101, -334.77322641,  -63.68921012,  -66.56291321,
        -63.89002839,  -48.28297386,  -32.17422733, -100.82682773,
        -33.65543035, -123.40225276, -110.03436199, -139.31897318,
        -27.53409263,  -23.64333478, -175.90106168,  -33.1034008 ,
        -44.30883494, -118.31044799,  -51.39885363,  -16.55721661])
 message: 'Optimization terminated successfully'
    nfev: 2997
     nit: 126
    njev: 125
  status: 0
 success: True
       x: array([-5.07546552e-08,  5.86332479e-08, -2.79038362e-09,  3.00901576e-06,
       -2.30603532e-07,  1.37931730e+00, -3.51510231e-07,  1.57145958e-09,
        1.18480549e-07,  4.06278598e-07,  6.28576504e-07, -1.49077231e-01,
       -1.95069781e-06,  5.71485102e-03,  6.41580157e-07, -2.61614229e-01,
        3.71897748e-08, -1.12794548e-06,  2.21291049e-06, -2.25305054e+00])

In [2]:
Pos[-1].round(3)

IndexError: list index out of range

In [3]:
plt.plot([p.x[i] * all_stocks[-1::].values[0][i] for i in range(len(p.x))])
plt.title("Argent investit dans chaque stocks")
plt.show()
print("L'argent est le plus investit dans l'action: "+symbols[np.argmax([p.x[i] * all_stocks[-1::].values[0][i] for i in range(len(p.x))])])

NameError: name 'plt' is not defined

In [44]:
# On a bien investit au total: Money=1000
np.dot(all_stocks[-1::].values[0],np.array(p.x).T)

1000.0000003452193

In [56]:
# Le returns avec une telle repartition de portefeuille est de:
np.sum(returns.mean()*p.x)

0.033959212018620635

In [53]:
print("Nombre d'action acheté pour chaque symbols: ")
r = pd.DataFrame([symbols,p.x])
for i in range(len(symbols)):
    print(symbols[i],p.x[i].round(3))

Nombre d'action acheté pour chaque symbols: 
SPY 1.607
QQQ 0.0
VGK 0.0
SCZ 0.0
EWJ 0.0
EEM 0.0
REM 0.0
VNQ 0.0
RWX 0.0
TIP 0.0
IEF 0.0
TLT 0.0
BWX 0.0
DBC 2.344
GLD 1.433
twtr 0.0
intc 0.0
tsm 0.0
VWUSX 0.0
Amc 0.0
